In [1]:
import os
import glob
import pandas as pd
from datetime import datetime
import time
#import numpy as np

In [2]:
variable_path_base = r'../data/variables'
variable_files_pattern = f'var-dia-*-fill.zip'
variable_datetime_format = '%Y-%m-%d' #'%Y-%m-%d'
variable_column_Valor = 'ValorFilled' 
variable_file_output = 'var-mes-{etiqueta}-fill.csv'

agg_valor = ['min','mean','median','max','count']

In [3]:
variable_path_base = variable_path_base.split('/')
variable_files = glob.glob(os.path.join(*variable_path_base, variable_files_pattern))
print(len(variable_files), variable_files)

6 ['..\\data\\variables\\var-dia-PTPM_CON-fill.zip', '..\\data\\variables\\var-dia-Q_MEDIA_D-fill.zip', '..\\data\\variables\\var-dia-Q_MN_D-fill.zip', '..\\data\\variables\\var-dia-Q_MX_D-fill.zip', '..\\data\\variables\\var-dia-TMN_CON-fill.zip', '..\\data\\variables\\var-dia-TMX_CON-fill.zip']


In [4]:
dtypes = {
    'CodigoEstacion':'string','Etiqueta':'category'#,'Fecha': 'string',
    #'IdParametro':'category','DescripcionSerie':'category','Frecuencia':'category','Grado':'category','Calificador':'category','NivelAprobacion':'category',
}

usecols = {
    'CodigoEstacion','Etiqueta', 'Fecha', variable_column_Valor
    #'IdParametro','DescripcionSerie','Frecuencia','Grado','Calificador','NivelAprobacion'
}

dateparse = lambda x: datetime.strptime(x, variable_datetime_format)

In [5]:
def groupVariableByMonth(f):
    start_time = time.time()
    
    print(f'Archivo: {f}')
    print(f'Leyendo h:{time.asctime()}...')
    df = pd.read_csv(f, dtype = dtypes, parse_dates = ['Fecha'], usecols = usecols, date_parser = dateparse)#, index_col = ['Fecha', 'CodigoEstacion'])
    print(f'**** r:{df.shape} h:{time.asctime()} t:{(time.time() - start_time) / 60} ****')
    
    df.rename(columns = {variable_column_Valor: 'Valor'}, inplace = True)
    
    etiqueta = df.loc[0,'Etiqueta']
    
    print(f'Etiqueta leída: {etiqueta}')
    
    print(f'Agrupando por mes...')
    dfm = df.groupby([df.CodigoEstacion,df.Etiqueta,df.Fecha.dt.to_period('M')]).Valor.agg(agg_valor).reset_index()
    print(f'**** r:{dfm.shape} h:{time.asctime()} t:{(time.time() - start_time) / 60} ****')
    print(dfm.head())
    
    p = os.path.join(*variable_path_base, variable_file_output.format(etiqueta = etiqueta))
    
    print(f'Guardando en {p}...')
    dfm.to_csv(p, index = False)
    print(f'**** h:{time.asctime()} t:{(time.time() - start_time) / 60} ****')

In [6]:
%%time
for f in variable_files:
    groupVariableByMonth(f)

Archivo: ..\data\variables\var-dia-PTPM_CON-fill.zip
Leyendo h:Tue Jun 30 18:27:07 2020...
**** r:(18694560, 4) h:Tue Jun 30 18:32:42 2020 t:5.578821221987407 ****
Etiqueta leída: PTPM_CON
Agrupando por mes...
**** r:(614220, 8) h:Tue Jun 30 18:32:53 2020 t:5.765304442246755 ****
  CodigoEstacion  Etiqueta    Fecha  min       mean  median    max  count
0       11010010  PTPM_CON  1991-01  0.0  11.958065    6.10   56.0     31
1       11010010  PTPM_CON  1991-02  0.0  20.017857    2.25  125.4     28
2       11010010  PTPM_CON  1991-03  0.0  21.890323    4.40   95.2     31
3       11010010  PTPM_CON  1991-04  0.0  10.826667    2.05   88.4     30
4       11010010  PTPM_CON  1991-05  0.0  23.612903   15.00   89.7     31
Guardando en ..\data\variables\var-mes-PTPM_CON-fill.csv...
**** h:Tue Jun 30 18:33:03 2020 t:5.919787796338399 ****
Archivo: ..\data\variables\var-dia-Q_MEDIA_D-fill.zip
Leyendo h:Tue Jun 30 18:33:03 2020...
**** r:(7886096, 4) h:Tue Jun 30 18:35:41 2020 t:2.636797722180684

In [7]:
#estacion = '13067020'
#year = 2005
#print(dfm.loc[(dfm.CodigoEstacion == estacion) & (dfm.Fecha.dt.year == year)].head(2))
#print(df[(df.CodigoEstacion == estacion) & (df.Fecha.dt.year == year)].head(2))